In [1]:
### Probando con datos
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ReadParquetYearMon16MM").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/07 17:41:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Mon Nov  7 17:42:00 2022


In [3]:
%%time
#spark job monitoring
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import sum, col, desc

esquema = StructType() \
      .add("Id",StringType(),True) \
      .add("fecNac",StringType(),True) \
      .add("sex",StringType(),True) \
      .add("ingresos",IntegerType(),True) \
      .add("partido",StringType(),True) \
      .add("edad",IntegerType(),True) \
      .add("mapa",StringType(),True)

#df = spark.read.parquet("./datos/fakePeopleParallel100M.parquet", sep=";", header = True, schema = esquema)

#df = spark.read.parquet("./datos/more/fakePeopleParallel100M.parquet/sex=*/partido=PP", sep=";", header = True, schema = esquema)

#df = spark.read.option("header", "false").option("schema", "esquema").option("mergeSchema", "true").option("sep", ";").parquet("./datos/more/fakePeopleParallel100M.parquet/sex=*/partido=P*")

"""
 * Lectura de un rango de meses de un parquet particionado por año/mes:
"""

def listPartitions (anoMesFinal, meses, path):
    lista = []
    mes = int(anoMesFinal[-2:]) + 1
    ano = int(anoMesFinal[:-3])
    
    print('Primer mes: '+anoMesFinal)
    
    for i in range(meses):
        mes = mes - 1
        if mes == 0:
            ano = ano - 1
            mes = 12
        if mes < 10: anoMes = str(ano)+'-0'+str(mes)
        else: anoMes = str(ano)+'-'+str(mes)
        #print('Desde la función: '+str(i+1)+' - '+anoMes)

        lista.append(path+anoMes)
    print('Último mes: '+anoMes)
    return lista

#df = spark.read.option("mergeSchema", True).option("schema", "esquema").opt/opt/workspace/datos/fecNac/fakePeople.parquetion("basePath", "file:///opt/workspace/datos/fecNac/fakePeople.parquet/").parquet('./datos/fecNac/fakePeople.parquet/yearMon=*')

#df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fecNac/fakePeople.parquet/")\
#               .format("parquet").load('./datos/fecNac/fakePeople.parquet/yearMon=1962-1*')


df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fecNac/fakePeople1600M.parquet/")\
               .format("parquet").load(listPartitions('2003-12', 1*12, './datos/fecNac/fakePeople1600M.parquet/yearMon='))
                                                                                                   
df.printSchema()

#df = spark.read.format('parquet').load('./datos/more/fakePeopleParallel100M.parquet/')
#df = df.where('sex=H')
print(df.count())
df.show()
df.explain(extended=True)
df.explain(mode='cost')


Primer mes: 2003-12
Último mes: 2003-01


root
 |-- Id: string (nullable = true)
 |-- fecNac: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- ingresos: integer (nullable = true)
 |-- partido: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- mapa: string (nullable = true)
 |-- yearMon: string (nullable = true)



19274200
+------------+----------+---+--------+-------+----+-------------+-------+
|          Id|    fecNac|sex|ingresos|partido|edad|         mapa|yearMon|
+------------+----------+---+--------+-------+----+-------------+-------+
|ID1881028601|2003-10-22|  M|   27300|     CS|  18|0-0-1-0-0-0-0|2003-10|
|ID1887311509|2003-10-31|  H|   13200|     PP|  18|1-1-0-0-0-1-1|2003-10|
|ID1892972910|2003-10-01|  H|   14800|     PP|  19|0-1-1-0-0-0-0|2003-10|
|ID1876920170|2003-10-11|  M|   28700|    VOX|  19|1-0-1-0-1-0-1|2003-10|
|ID1877935754|2003-10-18|  H|   18300|     CS|  18|1-1-0-0-1-0-0|2003-10|
|ID1882225202|2003-10-09|  M|   26600|     PP|  19|1-0-0-1-0-0-0|2003-10|
|ID1874145256|2003-10-31|  M|   74600|     CS|  18|1-0-1-0-0-1-1|2003-10|
|ID1881038945|2003-10-23|  M|   33400|   PSOE|  18|1-0-0-1-1-0-0|2003-10|
|ID1869880971|2003-10-20|  H|   21200|     PP|  18|0-1-0-0-0-1-1|2003-10|
|ID1886942769|2003-10-19|  M|   68500|     UP|  18|1-0-0-1-0-1-0|2003-10|
|ID1887323139|2003-10-22|  M|

In [4]:
print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Final: Mon Nov  7 17:46:57 2022
Hecho en 296.9921 segundo(s)


In [5]:
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

df.groupBy('yearMon').count().sort(desc('yearMon')).show(200)

print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Inicio: Mon Nov  7 17:46:57 2022


+-------+-------+
|yearMon|  count|
+-------+-------+
|2003-12|1606134|
|2003-11|1606203|
|2003-10|1605319|
|2003-09|1607650|
|2003-08|1606313|
|2003-07|1607364|
|2003-06|1605167|
|2003-05|1607874|
|2003-04|1603991|
|2003-03|1607177|
|2003-02|1605658|
|2003-01|1605350|
+-------+-------+

Final: Mon Nov  7 17:47:04 2022
Hecho en 6.7599 segundo(s)


In [6]:
esqPart = StructType() \
      .add("partido",StringType(),True) \
      .add("nombre",StringType(),True) \
      .add("lider",StringType(),True) \
      .add("fechaCreacion",StringType(),True)

dfPart = spark.read.csv("./datos/partidos.csv", sep=";", header = True, schema = esqPart)

dfPart.show()


+-------+--------------------+--------------------+-------------+
|partido|              nombre|               lider|fechaCreacion|
+-------+--------------------+--------------------+-------------+
|    VOX|     VOX España S.A.|    Santiago Abascal|   2015-01-01|
|     PP|     Partido Popular|Alberto Núñez Feijóo|   1986-10-31|
|   PSOE|Partido Socialist...|       Pedro Sánchez|   1930-12-25|
|     CS|          Ciudadanos|      Inés Arrimadas|   2008-04-18|
|     UP|      Unidas Podemos|        Yolanda Díaz|   2014-04-15|
|  Otros|          Ni se sabe|               Naide|   1975-11-20|
+-------+--------------------+--------------------+-------------+



In [7]:
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

df.join(dfPart, df.partido == dfPart.partido, "inner").show()

print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Inicio: Mon Nov  7 17:47:04 2022
+------------+----------+---+--------+-------+----+-------------+-------+-------+--------------------+--------------------+-------------+
|          Id|    fecNac|sex|ingresos|partido|edad|         mapa|yearMon|partido|              nombre|               lider|fechaCreacion|
+------------+----------+---+--------+-------+----+-------------+-------+-------+--------------------+--------------------+-------------+
|ID1881028601|2003-10-22|  M|   27300|     CS|  18|0-0-1-0-0-0-0|2003-10|     CS|          Ciudadanos|      Inés Arrimadas|   2008-04-18|
|ID1887311509|2003-10-31|  H|   13200|     PP|  18|1-1-0-0-0-1-1|2003-10|     PP|     Partido Popular|Alberto Núñez Feijóo|   1986-10-31|
|ID1892972910|2003-10-01|  H|   14800|     PP|  19|0-1-1-0-0-0-0|2003-10|     PP|     Partido Popular|Alberto Núñez Feijóo|   1986-10-31|
|ID1876920170|2003-10-11|  M|   28700|    VOX|  19|1-0-1-0-1-0-1|2003-10|    VOX|     VOX España S.A.|    Santiago Abascal|   2015-01-01|
|

In [8]:
spark.sparkContext.stop()
print('Sacabao')

Sacabao
